# Lab 02 - Azure OpenAI - Create a Completion

Given a prompt, the model will return one or more predicted completions, and can also return the probabilities of alternative tokens at each position.

In [15]:
import json
import openai
import os

### Setup Parameters


Here we will load the configurations from _config.json_ file to setup deployment name, openai api base, openai api key and openai api version.

In [16]:
# Setting up the deployment name
deployment_name = "text-davinci-003"

# This is set to `azure`
openai.api_type = "azure"
openai.api_version = "2022-12-01"

# The API key for your Azure OpenAI resource.
API_KEY = "<USE_YOUR_KEY>"
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = "https://openai-bootcamp-mma.openai.azure.com/"
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

In [32]:
def run_completion(prompt: str, deployment_name: str, temperaure=0.7, max_tokens=100, verbose=False):
    try:
        # Create a completion for the provided prompt and parameters
        # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
        completion = openai.Completion.create(
                        prompt=prompt,
                        temperature=temperaure,
                        max_tokens=max_tokens,
                        top_p=1,
                        frequency_penalty=0,
                        presence_penalty=0,
                        engine=deployment_name)

        # print the completion
        if (verbose):
            print(completion.choices[0].text.strip(" \n"))
        
        return completion.choices[0].text
        
        # Here indicating if the response is filtered
        if completion.choices[0].finish_reason == "content_filter":
            print("The generated content is filtered.")
            
    except openai.error.APIError as e:
        # Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")

    except openai.error.AuthenticationError as e:
        # Handle Authentication error here, e.g. invalid API key
        print(f"OpenAI API returned an Authentication Error: {e}")

    except openai.error.APIConnectionError as e:
        # Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")

    except openai.error.InvalidRequestError as e:
        # Handle connection error here
        print(f"Invalid Request Error: {e}")

    except openai.error.RateLimitError as e:
        # Handle rate limit error
        print(f"OpenAI API request exceeded rate limit: {e}")

    except openai.error.ServiceUnavailableError as e:
        # Handle Service Unavailable error
        print(f"Service Unavailable: {e}")

    except openai.error.Timeout as e:
        # Handle request timeout
        print(f"Request timed out: {e}")

In [18]:
# Give your prompt here
prompt = "Hello world"

run_completion(prompt, deployment_name)

"Hello! It's great to meet you."

# Exercises for several use cases  
1. Summarize Text  
2. Classify Text  
3. Generate New Product Names

## Summarize Text  
#### Challenge  
Summarize text by adding a 'tl;dr:' to the end of a text passage. Notice how the model understands how to perform a number of tasks with no additional instructions. You can experiment with more descriptive prompts than tl;dr to modify the model’s behavior and customize the summarization you receive(3).  

```
Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. 



Tl;dr
```

In [19]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"

# #Setting a few additional, typical parameters during API Call
# response = openai.Completion.create(
#   engine=model,
#   prompt=prompt,
#   temperature=0.7,
#   max_tokens=60,
#   top_p=1,
#   frequency_penalty=0,
#   presence_penalty=0,
#   stop=None)

# print(response)

run_completion(prompt=prompt, deployment_name=deployment_name)

': We show that scaling up language models improves task-agnostic, few-shot performance, and in some cases surpasses prior state-of-the-art fine-tuning approaches. This suggests that current NLP systems can better imitate human ability to quickly learn new tasks from only a few examples.'

## Classify Text  
#### Challenge  
Classify items into categories provided at inference time. In the following example we provide both the categories and the text to classify in the prompt(*playground_reference). 

```
Customer Inquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:

Classified category:
```


In [23]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"


run_completion(prompt=prompt, deployment_name=deployment_name)

'Hardware Support'

## Generate New Product Names
#### Challenge
Create product names from examples words. Here we include in the prompt information about the product we are going to generate names for. We also provide a similar example to show the pattern we wish to receive. We have also set the temperature value high to increase randomness and more innovative responses.

```
Product description: A home milkshake maker
Seed words: fast, healthy, compact.
Product names: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Product description: A pair of shoes that can fit any foot size.
Seed words: adaptable, fit, omni-fit.
```

In [24]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

run_completion(prompt=prompt, deployment_name=deployment_name)

'Product names: AdaptaFit, OmniFit, PerfectFit, FitPlus.'

## Generate product description

#### Challenge
Create product (fragnance) descriptions from ingredients. We have also set the temperature value high to increase randomness and more innovative responses.

Prompt:
```
Generate new product description from seed words.

Seed words: amber, plausible, fragrance, perfume, bergamot, cashmeran, black pepper
Product description:
```

In [27]:
prompt = '''
Generate new product description from seed words.

Seed words: amber, plausible, fragrance, perfume, bergamot, cashmeran, black pepper
Product description:
'''

run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=0.2)

'Introducing our new Amber Plausible Fragrance Perfume, a luxurious blend of bergamot, cashmeran, and black pepper. This exquisite scent is perfect for any occasion, and will leave you feeling confident and refreshed. With its warm and inviting aroma, this perfume is sure to become your signature scent.'

Adding additional information to prompt to change behavior.

Prompt:
```
As professional marketing specialist in retail and cosmetics company, generate new product description which is vibrant, uncommon, novel. 

Target group: young women between 15 and 25 year living in city. 

Seed words: amber, plausible, fragrance, perfume, bergamot, cashmeran, black pepper
Product description:
```

In [30]:
prompt = '''
As professional marketing specialist in retail and cosmetics company, generate new product description which is vibrant, uncommon, novel. 

Target group: young women between 15 and 25 year living in city. 

Seed words: amber, plausible, fragrance, perfume, bergamot, cashmeran, black pepper
Product description:
'''

run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=1.0, max_tokens=200)

'Introducing Amber Bliss – an enchanting perfume that will transform your aura! Infused with a sparkling blend of bergamot, cashmeran and black pepper, this highly plausible fragrance provides an energizing and vivacious sparkle to your everyday life. Soft and subtle, yet boldly vibrant, Amber Bliss is perfect for outgoing, young women who love to stand out and make their mark on the city!'

## Generate email responses

#### Challenge

Create email responses or communication using appropriate information and tone.

Prompt:
```
Okno údržby je naplánováno na 14. února od 14:00 do 15:00, linka bude funkční, ale očekávají se odstávky.
    Napište formální mail zákaznikům
    - Nabídněte pomoc v rámci info@operations.cz, ne telefonicky
    - Požádejte o prominutí
    - udělejte vtip o IT
    Mail:
```

In [39]:
prompt = '''
Okno údržby je naplánováno na 14. února od 14:00 do 15:00, linka bude funkční, ale očekávají se odstávky.
    
Napište formální mail zákaznikům
    - Nabídněte pomoc v rámci info@operations.cz, ne telefonicky
    - Požádejte o prominutí
    - udělejte vtip o IT

Mail:
'''

r = run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=1.0, max_tokens=500)
[print(x) for x in r.split("\n")]


Vážení zákazníci,

14. února od 14:00 do 15:00 se uskuteční naše pravidelné okno údržby. Během této doby bude linka funkční, i když může docházet ke krátkým odstávkám.

Omlouváme se za vzniklé potíže a nabízíme Vám, jako náhradu, informační kanál info@operations.cz.

IT je jako kuchař: občas se musíte smát špatnému oddílu.

S pozdravem,
Tým Operations


[None, None, None, None, None, None, None, None, None, None, None]

## Extract information

#### Challenge

Extract information from communication - the communication can be call transcription, email communication, etc. We can also define the desired output.

Prompt:
```
Extrahuj následující informace z telefonní komunikace níže:
    1. Důvod hovoru (klíč: reason)
    2. Jména účastníků hovoru jako pole (klíč: names)
    3. Krátké shrnutí hovoru (klíč: summary)
    Prosím o odpověď v JSON formát s použitím klíču výše.
    Formátuj výstup jako JSON object nazvaný "results" a výstup udělej v Markdown.
    Telefonní komunikace:
    Dobrý den, pane Štěpánek. Jsem z banky a potřebuji se s vámi poradit ohledně vašeho účtu.
    Máte nějaké dotazy?
    Štěpánek: Ano, pane bankéři. Mám dotaz ohledně výše úvěru.
    Bankéř: Ano, pane Štěpánek. Jaký úvěr máte na mysli?
    Štěpánek: Já bych chtěl zvýšit úvěr na bydlení.
    Bankéř: Ano, pane Štěpánek. Jakou máte v současné době výši úvěru?
    Štěpánek: Mám úvěr na bydlení v hodnotě 1 000 000 Kč.
    Bankéř: Ano, pane Štěpánek. Jakou výši byste chtěl zvýšit?
    Štěpánek: Chtěl bych zvýšit úvěr na bydlení na 1 500 000 Kč.
```

In [40]:
prompt = '''
    Extrahuj následující informace z telefonní komunikace níže:

    1. Důvod hovoru (klíč: reason)
    2. Jména účastníků hovoru jako pole (klíč: names)
    3. Krátké shrnutí hovoru (klíč: summary)

    Prosím o odpověď v JSON formát s použitím klíču výše.

    Formátuj výstup jako JSON object nazvaný "results" a výstup udělej v Markdown.

    Telefonní komunikace:
    Dobrý den, pane Štěpánek. Jsem z banky a potřebuji se s vámi poradit ohledně vašeho účtu.
    Máte nějaké dotazy?
    Štěpánek: Ano, pane bankéři. Mám dotaz ohledně výše úvěru.
    Bankéř: Ano, pane Štěpánek. Jaký úvěr máte na mysli?
    Štěpánek: Já bych chtěl zvýšit úvěr na bydlení.
    Bankéř: Ano, pane Štěpánek. Jakou máte v současné době výši úvěru?
    Štěpánek: Mám úvěr na bydlení v hodnotě 1 000 000 Kč.
    Bankéř: Ano, pane Štěpánek. Jakou výši byste chtěl zvýšit?
    Štěpánek: Chtěl bych zvýšit úvěr na bydlení na 1 500 000 Kč.
'''

r = run_completion(prompt=prompt, deployment_name=deployment_name, temperaure=1.0, max_tokens=500)
[print(x) for x in r.split("\n")]


Results: 
    {
        'reason': 'Konzultace ohledně úvěru na bydlení', 
        'names': ['Štěpánek', 'Bankéř'],
        'summary': 'Štěpánek požádal o zvýšení úvěru na bydlení z 1 000 000 Kč na 1 500 000 Kč.'
    }


[None, None, None, None, None, None, None]